In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
import json
import pprint

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from config import dbuser, dbpasswd, dburi, dbport, dbname

### Store CSV into DataFrame

In [2]:
csv_file = "KCPD_Crime_Data_2018_cleaned.csv"
kcmo_data_df = pd.read_csv(csv_file)
kcmo_data_df.head()

,#,Report_No,Reported_Date,Reported_Time,Offense,Description,Address,City,Zip_Code,Area,...,Involvement,Race,Sex,Age,Firearm_Used_Flag,Location,Lat_Lng,Lat,Lng,18325
0,2,180097829,12/20/2018,15:52,401,Aggravated Assault,500 BOOTH AV,KANSAS CITY,64124,EPD,...,SUS,W,M,18,N,"500 BOOTH AV\nKANSAS CITY 64124\n(39.105511, -...","39.105511, -94.482144",39.105511,-94.482144,NaN
1,4,180097829,12/20/2018,15:52,401,Aggravated Assault,500 BOOTH AV,KANSAS CITY,64124,EPD,...,SUS,P,M,18,N,"500 BOOTH AV\nKANSAS CITY 64124\n(39.105511, -...","39.105511, -94.482144",39.105511,-94.482144,NaN
2,9,180093115,12/3/2018,17:07,401,Aggravated Assault,4100 FLORA AV,KANSAS CITY,64110,CPD,...,SUS,W,F,18,Y,"4100 FLORA AV\nKANSAS CITY 64110\n(39.052161, ...","39.052161, -94.566533",39.052161,-94.566533,NaN
3,12,180091999,11/29/2018,14:11,401,Aggravated Assault,900 E 9 ST,KANSAS CITY,64108,CPD,...,SUS,W,M,18,N,"900 E 9 ST\nKANSAS CITY 64108\n(39.103158, -94...","39.103158, -94.57202",39.103150,-94.57202,NaN
4,17,180089325,11/18/2018,12:28,401,Aggravated Assault,6000 E 10 ST,KANSAS CITY,64126,EPD,...,SUS,U,M,18,Y,"6000 E 10 ST\nKANSAS CITY 64126\n(39.100277, -...","39.100277, -94.511176",39.100277,-94.511176,NaN


### Create new data with select columns

In [3]:
new_kcmo_data_df = kcmo_data_df[['Report_No', 'Reported_Date', 'Reported_Time', 'Offense', \
                                 'Description', 'Address', 'City', 'Zip_Code', 'Area', \
                                 'Invl_No', 'Involvement', 'Race', 'Sex', 'Age', \
                                 'Firearm_Used_Flag', 'Location', 'Lat_Lng', 'Lat', 'Lng']].copy()
new_kcmo_data_df.head()

,Report_No,Reported_Date,Reported_Time,Offense,Description,Address,City,Zip_Code,Area,Invl_No,Involvement,Race,Sex,Age,Firearm_Used_Flag,Location,Lat_Lng,Lat,Lng
0,180097829,12/20/2018,15:52,401,Aggravated Assault,500 BOOTH AV,KANSAS CITY,64124,EPD,2,SUS,W,M,18,N,"500 BOOTH AV\nKANSAS CITY 64124\n(39.105511, -...","39.105511, -94.482144",39.105511,-94.482144
1,180097829,12/20/2018,15:52,401,Aggravated Assault,500 BOOTH AV,KANSAS CITY,64124,EPD,1,SUS,P,M,18,N,"500 BOOTH AV\nKANSAS CITY 64124\n(39.105511, -...","39.105511, -94.482144",39.105511,-94.482144
2,180093115,12/3/2018,17:07,401,Aggravated Assault,4100 FLORA AV,KANSAS CITY,64110,CPD,1,SUS,W,F,18,Y,"4100 FLORA AV\nKANSAS CITY 64110\n(39.052161, ...","39.052161, -94.566533",39.052161,-94.566533
3,180091999,11/29/2018,14:11,401,Aggravated Assault,900 E 9 ST,KANSAS CITY,64108,CPD,2,SUS,W,M,18,N,"900 E 9 ST\nKANSAS CITY 64108\n(39.103158, -94...","39.103158, -94.57202",39.103150,-94.57202
4,180089325,11/18/2018,12:28,401,Aggravated Assault,6000 E 10 ST,KANSAS CITY,64126,EPD,1,SUS,U,M,18,Y,"6000 E 10 ST\nKANSAS CITY 64126\n(39.100277, -...","39.100277, -94.511176",39.100277,-94.511176


### Connect to local database

In [4]:
engine = create_engine(f"mysql://{dbuser}:{dbpasswd}@{dburi}:{dbport}/{dbname}")
Base.metadata.create_all(engine)

### Check for tables

In [5]:
engine.table_names()

['crime_data_2018', 'desc_summary', 'sex_summary']

### Use pandas to load csv converted DataFrame into database

In [6]:
new_kcmo_data_df.to_sql(name='crime_data_2018', con=engine, if_exists='replace', index=True)

In [7]:
pd.read_sql_query('select * from crime_data_2018', con=engine).head()

,index,Report_No,Reported_Date,Reported_Time,Offense,Description,Address,City,Zip_Code,Area,Invl_No,Involvement,Race,Sex,Age,Firearm_Used_Flag,Location,Lat_Lng,Lat,Lng
0,0,180097829,12/20/2018,15:52,401,Aggravated Assault,500 BOOTH AV,KANSAS CITY,64124,EPD,2,SUS,W,M,18,N,"500 BOOTH AV\nKANSAS CITY 64124\n(39.105511, -...","39.105511, -94.482144",39.105511,-94.482144
1,1,180097829,12/20/2018,15:52,401,Aggravated Assault,500 BOOTH AV,KANSAS CITY,64124,EPD,1,SUS,P,M,18,N,"500 BOOTH AV\nKANSAS CITY 64124\n(39.105511, -...","39.105511, -94.482144",39.105511,-94.482144
2,2,180093115,12/3/2018,17:07,401,Aggravated Assault,4100 FLORA AV,KANSAS CITY,64110,CPD,1,SUS,W,F,18,Y,"4100 FLORA AV\nKANSAS CITY 64110\n(39.052161, ...","39.052161, -94.566533",39.052161,-94.566533
3,3,180091999,11/29/2018,14:11,401,Aggravated Assault,900 E 9 ST,KANSAS CITY,64108,CPD,2,SUS,W,M,18,N,"900 E 9 ST\nKANSAS CITY 64108\n(39.103158, -94...","39.103158, -94.57202",39.103150,-94.57202
4,4,180089325,11/18/2018,12:28,401,Aggravated Assault,6000 E 10 ST,KANSAS CITY,64126,EPD,1,SUS,U,M,18,Y,"6000 E 10 ST\nKANSAS CITY 64126\n(39.100277, -...","39.100277, -94.511176",39.100277,-94.511176


### Confirm data and write full dataset to a file in JSON format

In [8]:
temp_json = new_kcmo_data_df.head(3).to_json(orient='records')
new_kcmo_data_df.to_json(orient='records', path_or_buf='kcpd_crime.json')

### Write the temp data to confirm it worked

In [9]:
parsed = json.loads(temp_json)
print(json.dumps(parsed, indent=4, sort_keys=True))
# pprint(temp_json)



[
    {
        "Address": "500  BOOTH AV",
        "Age": 18,
        "Area": "EPD",
        "City": "KANSAS CITY",
        "Description": "Aggravated Assault",
        "Firearm_Used_Flag": "N",
        "Invl_No": 2,
        "Involvement": "SUS",
        "Lat": 39.105511,
        "Lat_Lng": "39.105511, -94.482144",
        "Lng": "-94.482144",
        "Location": "500 BOOTH AV\nKANSAS CITY 64124\n(39.105511, -94.482144)",
        "Offense": 401,
        "Race": "W",
        "Report_No": 180097829,
        "Reported_Date": "12/20/2018",
        "Reported_Time": "15:52",
        "Sex": "M",
        "Zip_Code": 64124
    },
    {
        "Address": "500  BOOTH AV",
        "Age": 18,
        "Area": "EPD",
        "City": "KANSAS CITY",
        "Description": "Aggravated Assault",
        "Firearm_Used_Flag": "N",
        "Invl_No": 1,
        "Involvement": "SUS",
        "Lat": 39.105511,
        "Lat_Lng": "39.105511, -94.482144",
        "Lng": "-94.482144",
        "Location": "500 

## Create Description Summary Info

In [10]:
new_kcmo_data_df.columns

Index(['Report_No', 'Reported_Date', 'Reported_Time', 'Offense', 'Description',
       'Address', 'City', 'Zip_Code', 'Area', 'Invl_No', 'Involvement', 'Race',
       'Sex', 'Age', 'Firearm_Used_Flag', 'Location', 'Lat_Lng', 'Lat', 'Lng'],
      dtype='object')

In [11]:
# Using GroupBy in order to separate the data into fields according to "Description"
grouped_desc_df = new_kcmo_data_df.groupby(['Description'])

# In order to be visualized, a data function must be used...
desc_counts = grouped_desc_df['Description'].count()
# desc_counts.set_index('Description', inplace=True)
desc_counts.head()


Description
Aggravated Assault    1725
Animal Cruelty           2
Armed Robbery          317
Arson                   33
Attempt Suicide          6
Name: Description, dtype: int64

In [12]:
desc_count_df = grouped_desc_df

In [13]:
total_crimes = new_kcmo_data_df['Description'].count()
total_crimes

18324

In [14]:
# Creating a new DataFrame using both duration, count and the calculated percent
desc_count_df = pd.DataFrame({ "Description": desc_counts.index, 
                              "Count": desc_counts, 
                              "Percent": desc_counts / total_crimes * 100})
desc_count_df.head()


,Description,Count,Percent
Description,,,
Aggravated Assault,Aggravated Assault,1725,9.413883
Animal Cruelty,Animal Cruelty,2,0.010915
Armed Robbery,Armed Robbery,317,1.729972
Arson,Arson,33,0.180092
Attempt Suicide,Attempt Suicide,6,0.032744


### Use pandas to load DataFrame into database

In [15]:
desc_count_df.columns

Index(['Description', 'Count', 'Percent'], dtype='object')

In [16]:
# desc_count_df.reset_index()
# desc_count_df.head()

In [17]:
# Create MySQL table and write Desc_count DF to this MySQL table
desc_count_df.to_sql(name='desc_summary', con=engine, if_exists='replace', index=False)
desc_count_df.head()

,Description,Count,Percent
Description,,,
Aggravated Assault,Aggravated Assault,1725,9.413883
Animal Cruelty,Animal Cruelty,2,0.010915
Armed Robbery,Armed Robbery,317,1.729972
Arson,Arson,33,0.180092
Attempt Suicide,Attempt Suicide,6,0.032744


### Write the temp data to confirm it worked

In [18]:
temp_desc_json = desc_count_df.head(3).to_json(orient='records')
desc_count_df.to_json(orient='records', path_or_buf='desc_summary.json')

In [19]:
parsed = json.loads(temp_desc_json)
print(json.dumps(parsed, indent=4, sort_keys=True))


[
    {
        "Count": 1725,
        "Description": "Aggravated Assault",
        "Percent": 9.4138834316
    },
    {
        "Count": 2,
        "Description": "Animal Cruelty",
        "Percent": 0.0109146475
    },
    {
        "Count": 317,
        "Description": "Armed Robbery",
        "Percent": 1.7299716219
    }
]


## Create Sex Summary Info

In [20]:
# Using GroupBy in order to separate the data into fields according to "Description"
grouped_sex_df = new_kcmo_data_df.groupby(['Sex'])

# In order to be visualized, a data function must be used...
sex_counts = grouped_sex_df['Sex'].count()
# desc_counts.set_index('Description', inplace=True)
sex_counts.head()


Sex
F     5360
M    12874
U       90
Name: Sex, dtype: int64

In [21]:
# NOTE:
# total_crimes = new_kcmo_data_df['Description'].count()
# total_crimes = 18324

In [22]:
# Creating a new DataFrame using both duration, count and the calculated percent
sex_count_df = pd.DataFrame({ "Sex": sex_counts.index, 
                              "Count": sex_counts, 
                              "Percent": sex_counts / total_crimes * 100})
sex_count_df.head()


,Sex,Count,Percent
Sex,,,
F,F,5360,29.251255
M,M,12874,70.257586
U,U,90,0.491159


### Use pandas to load DataFrame into database

In [23]:
sex_count_df.columns

Index(['Sex', 'Count', 'Percent'], dtype='object')

In [24]:
# Create MySQL table and write Desc_count DF to this MySQL table
sex_count_df.to_sql(name='sex_summary', con=engine, if_exists='replace', index=False)
sex_count_df.head()

,Sex,Count,Percent
Sex,,,
F,F,5360,29.251255
M,M,12874,70.257586
U,U,90,0.491159


### Write the temp data to confirm it worked

In [25]:
temp_sex_json = sex_count_df.head(3).to_json(orient='records')
sex_count_df.to_json(orient='records', path_or_buf='sex_summary.json')

In [26]:
parsed = json.loads(temp_sex_json)
print(json.dumps(parsed, indent=4, sort_keys=True))


[
    {
        "Count": 5360,
        "Percent": 29.2512551845,
        "Sex": "F"
    },
    {
        "Count": 12874,
        "Percent": 70.25758568,
        "Sex": "M"
    },
    {
        "Count": 90,
        "Percent": 0.4911591356,
        "Sex": "U"
    }
]


## Create Age Summary Info

In [27]:
df = pd.DataFrame(new_kcmo_data_df)
df['Percent'] = 0
df

,Report_No,Reported_Date,Reported_Time,Offense,Description,Address,City,Zip_Code,Area,Invl_No,Involvement,Race,Sex,Age,Firearm_Used_Flag,Location,Lat_Lng,Lat,Lng,Percent
0,180097829,12/20/2018,15:52,401,Aggravated Assault,500 BOOTH AV,KANSAS CITY,64124,EPD,2,SUS,W,M,18,N,"500 BOOTH AV\nKANSAS CITY 64124\n(39.105511, -...","39.105511, -94.482144",39.105511,-94.482144,0
1,180097829,12/20/2018,15:52,401,Aggravated Assault,500 BOOTH AV,KANSAS CITY,64124,EPD,1,SUS,P,M,18,N,"500 BOOTH AV\nKANSAS CITY 64124\n(39.105511, -...","39.105511, -94.482144",39.105511,-94.482144,0
2,180093115,12/3/2018,17:07,401,Aggravated Assault,4100 FLORA AV,KANSAS CITY,64110,CPD,1,SUS,W,F,18,Y,"4100 FLORA AV\nKANSAS CITY 64110\n(39.052161, ...","39.052161, -94.566533",39.052161,-94.566533,0
3,180091999,11/29/2018,14:11,401,Aggravated Assault,900 E 9 ST,KANSAS CITY,64108,CPD,2,SUS,W,M,18,N,"900 E 9 ST\nKANSAS CITY 64108\n(39.103158, -94...","39.103158, -94.57202",39.103150,-94.57202,0
4,180089325,11/18/2018,12:28,401,Aggravated Assault,6000 E 10 ST,KANSAS CITY,64126,EPD,1,SUS,U,M,18,Y,"6000 E 10 ST\nKANSAS CITY 64126\n(39.100277, -...","39.100277, -94.511176",39.100277,-94.511176,0
5,180086401,11/7/2018,13:37,401,Aggravated Assault,4900 SWOPE PK,KANSAS CITY,64130,MPD,1,SUS,B,M,18,Y,"4900 SWOPE PK\nKANSAS CITY 64130\n(39.03717, -...","39.03717, -94.54451",39.037170,-94.54451,0
6,180070848,9/13/2018,8:17,401,Aggravated Assault,1200 E TRUMAN RD,KANSAS CITY,64106,CPD,1,ARR,B,F,18,N,1200 E TRUMAN RD\nKANSAS CITY 64106\n(39.09529...,"39.095297, -94.568066",39.095297,-94.568066,0
7,180072347,9/18/2018,20:08,401,Aggravated Assault,9700 MARION PARK DR,KANSAS CITY,64137,SPD,1,SUS,B,M,18,Y,9700 MARION PARK DR\nKANSAS CITY 64137\n(38.95...,"38.950149, -94.536077",38.950149,-94.536077,0
8,180073105,9/21/2018,3:00,401,Aggravated Assault,4500 SOUTH BENTON AV,KANSAS CITY,64130,EPD,1,SUS,B,F,18,N,4500 SOUTH BENTON AV\nKANSAS CITY 64130\n(39.0...,"39.044398, -94.550928",39.044398,-94.550928,0
9,180072410,9/18/2018,18:30,401,Aggravated Assault,5100 E 24 ST,KANSAS CITY,64127,EPD,2,SUS,B,M,18,N,"5100 E 24 ST\nKANSAS CITY 64127\n(39.081734, -...","39.081734, -94.522883",39.081734,-94.522883,0


In [28]:
# Create the bins in which Data will be held
# Bins are 0, 60, 70, 80, 90, 100
bins = [0, 20, 30, 40, 50, 60, 70, 80, 90, 100]

# Create the names for the four bins
group_names = ["<20", "20-30", "30-40", "40-50", "50-60", "60-70", "70-80", "80-90", "90-100"]

In [29]:
# Convert the Age to a number
df["Age"] = pd.to_numeric(df["Age"])

In [30]:
df["Age_Summary"] = pd.cut(df["Age"], bins, labels=group_names)
df

,Report_No,Reported_Date,Reported_Time,Offense,Description,Address,City,Zip_Code,Area,Invl_No,...,Race,Sex,Age,Firearm_Used_Flag,Location,Lat_Lng,Lat,Lng,Percent,Age_Summary
0,180097829,12/20/2018,15:52,401,Aggravated Assault,500 BOOTH AV,KANSAS CITY,64124,EPD,2,...,W,M,18,N,"500 BOOTH AV\nKANSAS CITY 64124\n(39.105511, -...","39.105511, -94.482144",39.105511,-94.482144,0,<20
1,180097829,12/20/2018,15:52,401,Aggravated Assault,500 BOOTH AV,KANSAS CITY,64124,EPD,1,...,P,M,18,N,"500 BOOTH AV\nKANSAS CITY 64124\n(39.105511, -...","39.105511, -94.482144",39.105511,-94.482144,0,<20
2,180093115,12/3/2018,17:07,401,Aggravated Assault,4100 FLORA AV,KANSAS CITY,64110,CPD,1,...,W,F,18,Y,"4100 FLORA AV\nKANSAS CITY 64110\n(39.052161, ...","39.052161, -94.566533",39.052161,-94.566533,0,<20
3,180091999,11/29/2018,14:11,401,Aggravated Assault,900 E 9 ST,KANSAS CITY,64108,CPD,2,...,W,M,18,N,"900 E 9 ST\nKANSAS CITY 64108\n(39.103158, -94...","39.103158, -94.57202",39.103150,-94.57202,0,<20
4,180089325,11/18/2018,12:28,401,Aggravated Assault,6000 E 10 ST,KANSAS CITY,64126,EPD,1,...,U,M,18,Y,"6000 E 10 ST\nKANSAS CITY 64126\n(39.100277, -...","39.100277, -94.511176",39.100277,-94.511176,0,<20
5,180086401,11/7/2018,13:37,401,Aggravated Assault,4900 SWOPE PK,KANSAS CITY,64130,MPD,1,...,B,M,18,Y,"4900 SWOPE PK\nKANSAS CITY 64130\n(39.03717, -...","39.03717, -94.54451",39.037170,-94.54451,0,<20
6,180070848,9/13/2018,8:17,401,Aggravated Assault,1200 E TRUMAN RD,KANSAS CITY,64106,CPD,1,...,B,F,18,N,1200 E TRUMAN RD\nKANSAS CITY 64106\n(39.09529...,"39.095297, -94.568066",39.095297,-94.568066,0,<20
7,180072347,9/18/2018,20:08,401,Aggravated Assault,9700 MARION PARK DR,KANSAS CITY,64137,SPD,1,...,B,M,18,Y,9700 MARION PARK DR\nKANSAS CITY 64137\n(38.95...,"38.950149, -94.536077",38.950149,-94.536077,0,<20
8,180073105,9/21/2018,3:00,401,Aggravated Assault,4500 SOUTH BENTON AV,KANSAS CITY,64130,EPD,1,...,B,F,18,N,4500 SOUTH BENTON AV\nKANSAS CITY 64130\n(39.0...,"39.044398, -94.550928",39.044398,-94.550928,0,<20
9,180072410,9/18/2018,18:30,401,Aggravated Assault,5100 E 24 ST,KANSAS CITY,64127,EPD,2,...,B,M,18,N,"5100 E 24 ST\nKANSAS CITY 64127\n(39.081734, -...","39.081734, -94.522883",39.081734,-94.522883,0,<20


In [31]:
# Creating a group based off of the bins
df = df.groupby("Age_Summary").count().reset_index()
df2 = df

In [32]:
df2

,Age_Summary,Report_No,Reported_Date,Reported_Time,Offense,Description,Address,City,Zip_Code,Area,...,Involvement,Race,Sex,Age,Firearm_Used_Flag,Location,Lat_Lng,Lat,Lng,Percent
0,<20,2042,2042,2042,2042,2042,2042,2042,2042,2026,...,2042,2042,2042,2042,2042,2042,2042,2042,2042,2042
1,20-30,6554,6554,6554,6554,6554,6554,6554,6554,6511,...,6554,6554,6554,6554,6554,6554,6554,6554,6554,6554
2,30-40,4947,4947,4947,4947,4947,4947,4947,4947,4916,...,4947,4947,4947,4947,4947,4947,4947,4947,4947,4947
3,40-50,2577,2577,2577,2577,2577,2577,2577,2577,2565,...,2577,2577,2577,2577,2577,2577,2577,2577,2577,2577
4,50-60,1642,1642,1642,1642,1642,1642,1642,1642,1636,...,1642,1642,1642,1642,1642,1642,1642,1642,1642,1642
5,60-70,394,394,394,394,394,394,394,394,392,...,394,394,394,394,394,394,394,394,394,394
6,70-80,78,78,78,78,78,78,78,78,75,...,78,78,78,78,78,78,78,78,78,78
7,80-90,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
8,90-100,87,87,87,87,87,87,87,87,86,...,87,87,87,87,87,87,87,87,87,87


In [33]:
df2 = df2.rename(columns={"Age":"Count"})
df2.head()

,Age_Summary,Report_No,Reported_Date,Reported_Time,Offense,Description,Address,City,Zip_Code,Area,...,Involvement,Race,Sex,Count,Firearm_Used_Flag,Location,Lat_Lng,Lat,Lng,Percent
0,<20,2042,2042,2042,2042,2042,2042,2042,2042,2026,...,2042,2042,2042,2042,2042,2042,2042,2042,2042,2042
1,20-30,6554,6554,6554,6554,6554,6554,6554,6554,6511,...,6554,6554,6554,6554,6554,6554,6554,6554,6554,6554
2,30-40,4947,4947,4947,4947,4947,4947,4947,4947,4916,...,4947,4947,4947,4947,4947,4947,4947,4947,4947,4947
3,40-50,2577,2577,2577,2577,2577,2577,2577,2577,2565,...,2577,2577,2577,2577,2577,2577,2577,2577,2577,2577
4,50-60,1642,1642,1642,1642,1642,1642,1642,1642,1636,...,1642,1642,1642,1642,1642,1642,1642,1642,1642,1642


In [34]:
df2 = df2[['Age_Summary', 'Count', 'Percent']]

In [35]:
percent = df2["Count"]/total_crimes
#df2.loc['Percent'] = percent
df2["Percent"] = percent


In [36]:
df2

,Age_Summary,Count,Percent
0,<20,2042,0.111439
1,20-30,6554,0.357673
2,30-40,4947,0.269974
3,40-50,2577,0.140635
4,50-60,1642,0.089609
5,60-70,394,0.021502
6,70-80,78,0.004257
7,80-90,3,0.000164
8,90-100,87,0.004748


In [37]:
# # Using GroupBy in order to separate the data into fields according to "Description"
# age_counts = df

# # In order to be visualized, a data function must be used...
# #age_counts = grouped_age_df
# # desc_counts.set_index('Description', inplace=True)
# age_counts.head()

In [38]:
# # Creating a new DataFrame using both duration, count and the calculated percent
# df = pd.DataFrame({ "Age_Summary": df.index, 
#                               "Age": age_counts)
# df.head()

In [39]:
df3 = df2.reset_index()
df3

,index,Age_Summary,Count,Percent
0,0,<20,2042,0.111439
1,1,20-30,6554,0.357673
2,2,30-40,4947,0.269974
3,3,40-50,2577,0.140635
4,4,50-60,1642,0.089609
5,5,60-70,394,0.021502
6,6,70-80,78,0.004257
7,7,80-90,3,0.000164
8,8,90-100,87,0.004748


In [40]:
# Create MySQL table and write Desc_count DF to this MySQL table
df2.to_sql(name='age_summary', con=engine, if_exists='replace', index=True)
df2

,Age_Summary,Count,Percent
0,<20,2042,0.111439
1,20-30,6554,0.357673
2,30-40,4947,0.269974
3,40-50,2577,0.140635
4,50-60,1642,0.089609
5,60-70,394,0.021502
6,70-80,78,0.004257
7,80-90,3,0.000164
8,90-100,87,0.004748


### Write the temp data to confirm it worked

In [41]:
temp_age_json = df2.to_json(orient='records')
df2.to_json(orient='records', path_or_buf='age_summary.json')

In [42]:
parsed = json.loads(temp_age_json)
print(json.dumps(parsed, indent=4, sort_keys=True))


[
    {
        "Age_Summary": "<20",
        "Count": 2042,
        "Percent": 0.1114385505
    },
    {
        "Age_Summary": "20-30",
        "Count": 6554,
        "Percent": 0.3576729972
    },
    {
        "Age_Summary": "30-40",
        "Count": 4947,
        "Percent": 0.2699738048
    },
    {
        "Age_Summary": "40-50",
        "Count": 2577,
        "Percent": 0.1406352325
    },
    {
        "Age_Summary": "50-60",
        "Count": 1642,
        "Percent": 0.0896092556
    },
    {
        "Age_Summary": "60-70",
        "Count": 394,
        "Percent": 0.0215018555
    },
    {
        "Age_Summary": "70-80",
        "Count": 78,
        "Percent": 0.0042567125
    },
    {
        "Age_Summary": "80-90",
        "Count": 3,
        "Percent": 0.0001637197
    },
    {
        "Age_Summary": "90-100",
        "Count": 87,
        "Percent": 0.0047478716
    }
]


In [43]:
# df = pd.DataFrame({ "Age_Summary": sex_counts.index, 
#                               "Count": sex_counts, 
#                               "Percent": sex_counts / total_crimes * 100})
# df